<a href="https://colab.research.google.com/github/ArindamRoy23/Twitter_sentiment_analysis/blob/main/Twitter_sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install emoji
import numpy as np
import pandas as pd
import nltk
from google.colab import drive
import emoji
# from utils import *
from sklearn.preprocessing import LabelEncoder
import re
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !pip install --upgrade kaggle
# # Mount your Google Drive.
# kaggle_creds_path = r'/content/drive/MyDrive/kaggle_key.json/'
# !cp "/content/drive/My Drive/kaggle_key.json" ~/.kaggle/kaggle.json
# !chmod 600 ~/.kaggle/kaggle.json
# !ls ~/.kaggle

In [3]:
!mkdir /content/data
file_path = r'/content/drive/MyDrive/twitter_data.zip'
!unzip /content/drive/MyDrive/twitter_data.zip -d /content/data

mkdir: cannot create directory ‘/content/data’: File exists
Archive:  /content/drive/MyDrive/twitter_data.zip
replace /content/data/twitter_training.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/data/twitter_validation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [4]:
train_data = pd.read_csv(r'/content/data/twitter_training.csv', header = None)
val_data = pd.read_csv(r'/content/data/twitter_validation.csv', header = None)

In [5]:
train_data.shape
val_data.shape

(1000, 4)

# Utils

In [6]:
# Move to utils
def rename_headers(df, rename_dict = {0: 'tweet_id', 1: 'tag' , 2: 'label', 3:'text'}):
  df = df.rename(columns = rename_dict)
  return(df)

def select_columns(df, columns_to_keep = ['label','text']):
  df = df[columns_to_keep]
  return(df)

def trim_garbage(df):
  df = df.dropna()
  return(df)

def tweet_cleaner(tweet, words):

    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.EMOJI_DATA) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if w.lower() in words or not w.isalpha())
    return tweet


# Data Preprocesing

In [7]:
train_data = rename_headers(train_data)
val_data = rename_headers(val_data)
train_data = select_columns(train_data)
val_data = select_columns(val_data)
train_data = trim_garbage(train_data)
val_data = trim_garbage(val_data)


In [8]:
# Encoding labels
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['label'])
val_data['label'] = le.transform(val_data['label'])

In [9]:
# Tweet cleaning
nltk.download('words')
words = set(nltk.corpus.words.words())
train_data.text = train_data.text.apply(lambda x: tweet_cleaner(x, words))
val_data.text = val_data.text.apply(lambda x: tweet_cleaner(x, words))

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [10]:
train_data

,label,text
0,3,"getting on and i will murder you all ,"
1,3,"I am coming to the and I will kill you all ,"
2,3,"getting on and i will kill you all ,"
3,3,"coming on and i will murder you all ,"
4,3,"getting on 2 and i will murder you me all ,"
...,...,...
74677,3,Just that the partition of my Mac is like 6 be...
74678,3,Just that my Mac window partition is 6 behind ...
74679,3,Just the partition of my Mac is now 6 behind o...
74680,3,Just between the partition of my Mac is like b...


In [11]:
!pip install transformers
import transformers
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax
import csv
import urllib.request

In [14]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

In [15]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [16]:
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

In [18]:
# Test_run
text = "I dislike this so very much"
# text = preprocess(text)
text = tweet_cleaner(text, words)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
# score_generator(tweet_cleaner(text, words))

In [19]:
mapping_dict = {0 : 'Negative', 1 : 'Neutral', 2 : 'Positive'}

In [21]:
from transformers import TextClassificationPipeline
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [26]:
pipe(["I love this movie!"])

[[{'label': 'LABEL_0', 'score': 0.0022000870667397976},
  {'label': 'LABEL_1', 'score': 0.005600492004305124},
  {'label': 'LABEL_2', 'score': 0.9921994805335999}]]

In [ ]:
preds = pipe(train_data.text.to_list())

In [ ]:
print(preds[1:3])